In [1]:
# main_script.py
import os
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from diffusers import StableDiffusionXLPipeline, StableVideoDiffusionPipeline, DiffusionPipeline
from diffusers.utils import load_image, export_to_video
from TTS.api import TTS # Coqui TTS
from moviepy import VideoFileClip, AudioFileClip, concatenate_videoclips, TextClip, CompositeVideoClip
import gc # Garbage collection

In [2]:
# --- 0. CONFIGURATION ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
OUTPUT_DIR = "instagram_content"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# LLM paths (example, download these first)
LLM_MODEL_ID = "HuggingFaceH4/zephyr-7b-beta"
# TTS model (XTTSv2 example)
TTS_MODEL_ID = "tts_models/multilingual/multi-dataset/xtts_v2" # or "tts_models/en/ljspeech/tacotron2-DDC"
# T2I model
T2I_MODEL_ID = "stabilityai/stable-diffusion-xl-base-1.0"
T2I_REFINER_ID = "stabilityai/stable-diffusion-xl-refiner-1.0"
# I2V model (SVD)
I2V_MODEL_ID = "stabilityai/stable-video-diffusion-img2vid-xt"
# T2V model (alternative)
T2V_MODEL_ID = "damo-vilab/text-to-video-ms-1.7b"


In [3]:
# --- 1. INITIALIZE MODELS (Load only when needed or keep loaded if VRAM allows) ---
def load_llm():
    print("Loading LLM...")
    tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_ID)
    # Ensure pad_token is set if model doesn't have one; often same as eos_token for CausalLMs
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL_ID,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    # We won't use the pipeline directly for chat templating, but generate manually
    return model, tokenizer

    
def load_tts():
    print("Loading TTS model...")
    # Make sure you have the model downloaded or it will download on first run
    # For XTTSv2, you might need to specify a speaker_wav for voice cloning
    return TTS(model_name=TTS_MODEL_ID, progress_bar=True).to(DEVICE)

def load_t2i_pipeline():
    print("Loading T2I pipeline (SDXL)...")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        T2I_MODEL_ID, torch_dtype=torch.float16, variant="fp16", use_safetensors=True
    ).to(DEVICE)
    # Optional: Load refiner if you want to use it
    # refiner = DiffusionPipeline.from_pretrained(
    #     T2I_REFINER_ID, text_encoder_2=pipe.text_encoder_2, vae=pipe.vae,
    #     torch_dtype=torch.float16, use_safetensors=True, variant="fp16"
    # ).to(DEVICE)
    # return pipe, refiner
    return pipe, None # Simpler for now

def load_i2v_pipeline(): # SVD
    print("Loading I2V pipeline (SVD)...")
    pipe = StableVideoDiffusionPipeline.from_pretrained(
        I2V_MODEL_ID, torch_dtype=torch.float16, variant="fp16"
    ).to(DEVICE)
    pipe.enable_model_cpu_offload() # If VRAM is tight with other models
    return pipe

def load_t2v_pipeline(): # ModelScope Text-to-Video
    print("Loading T2V pipeline (ModelScope)...")
    # Use DiffusionPipeline as per the documentation for damo-vilab/text-to-video-ms-1.7b
    pipe = DiffusionPipeline.from_pretrained(
        T2V_MODEL_ID, torch_dtype=torch.float16, variant="fp16"
    ).to(DEVICE)
    # The .enable_model_cpu_offload() should still work if the loaded pipe supports it
    # For ModelScope, this is standard.
    pipe.enable_model_cpu_offload()
    return pipe

In [4]:
# --- UTILITY TO CLEAR VRAM ---
def clear_vram(*models_or_pipelines):
    for item in models_or_pipelines:
        if hasattr(item, 'cpu') and callable(getattr(item, 'cpu')):
            item.cpu() # If it's a pipeline/model with a .cpu() method
        elif hasattr(item, 'model') and hasattr(item.model, 'cpu') and callable(getattr(item.model, 'cpu')):
            item.model.cpu()
    del models_or_pipelines
    torch.cuda.empty_cache()
    gc.collect()
    print("VRAM cleared and memory collected.")


In [5]:
# --- 2. TEXT GENERATION ---
def generate_script_and_prompts_with_chat_template(topic, model, tokenizer):
    print(f"Generating script and prompts for topic (chat template): {topic}")

    # Define the messages for the chat template
    messages = [
        {
            "role": "system",
            "content": "You are an AI assistant creating content for an Instagram Reel. Format your response clearly with 'Narration Script:', 'Visual Prompts:', and 'Hashtags:' sections. Only output these sections."
        },
        {
            "role": "user",
            "content": f"""
            Create content for an Instagram Reel about "{topic}".
            The Reel should be engaging and around 15-30 seconds long.
            Provide the following:
            1. Narration Script: A short, punchy narration. Break it into 2-3 short sentences or scenes.
            2. Visual Prompts: For each sentence/scene in the narration, provide a detailed visual prompt suitable for an image or video generation model. Describe the scene, style, colors, and mood.
            3. Hashtags: 3-5 relevant hashtags.

            Example Format:
            Narration Script:
            Scene 1: [Sentence 1 for narration]
            Scene 2: [Sentence 2 for narration]

            Visual Prompts:
            Scene 1: [Visual prompt for sentence 1]
            Scene 2: [Visual prompt for sentence 2]

            Hashtags:
            #tag1 #tag2 #tag3
            """
        }
    ]

    # Apply the chat template
    # add_generation_prompt=True tells the tokenizer to add the tokens that signal the start of the assistant's turn
    tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)
    
    # Generate response
    # Ensure model.config.pad_token_id is set if different from eos_token_id
    # For Zephyr, eos_token_id is usually sufficient for pad_token_id in generation
    generation_kwargs = {
        "input_ids": tokenized_chat,
        "max_new_tokens": 512,
        "do_sample": True, # Enable sampling
        "top_k": 50,
        "top_p": 0.95,
        "temperature": 0.7,
        "pad_token_id": tokenizer.eos_token_id # Crucial for open-ended generation
    }
    
    outputs = model.generate(**generation_kwargs)
    
    # Decode only the generated part (after the input sequence)
    # outputs[0] is the full sequence (input + generated)
    # tokenized_chat.shape[-1] is the length of the input
    decoded_output = tokenizer.decode(outputs[0][tokenized_chat.shape[-1]:], skip_special_tokens=True)

    print("LLM Response (model generated part only with chat template):\n", decoded_output)
    
    model_generated_part = decoded_output.strip()
    
    # --- Your existing parsing logic using `model_generated_part` (same as in Option 1's parsing block) ---
    # ... (copy the parsing block from Option 1, starting with `response_to_parse = model_generated_part` )
    # For brevity, I'll just outline it:
    parsed_narration = []
    parsed_visuals = []
    parsed_hashtags = ""

    try:
        narration_script_content = model_generated_part.split("Narration Script:")[1].split("Visual Prompts:")[0]
        parsed_narration = [s.split(":", 1)[1].strip() for s in narration_script_content.splitlines() if "Scene" in s and ":" in s]

        visual_prompts_content = model_generated_part.split("Visual Prompts:")[1].split("Hashtags:")[0]
        parsed_visuals = [s.split(":", 1)[1].strip() for s in visual_prompts_content.splitlines() if "Scene" in s and ":" in s]
        
        if "Hashtags:" in model_generated_part:
            hashtags_content = model_generated_part.split("Hashtags:")[1]
            parsed_hashtags = hashtags_content.strip()
        else:
            parsed_hashtags = "#GeneratedContent"
            
    except IndexError:
        print("Error: Could not find expected section markers (Narration Script:, Visual Prompts:, Hashtags:) in LLM response.")
        # (Keep fallback logic as before)

    if not parsed_narration or not parsed_visuals:
        print("Error: Could not parse LLM response properly (chat template). Using fallback.")
        parsed_narration = [f"A short segment about {topic}.", "Concluding thoughts on the future."]
        parsed_visuals = [
            f"Cinematic shot of {topic}, vibrant colors, high detail, trending on artstation.",
            f"Abstract representation of {topic}, thought-provoking, professional."
        ]
        parsed_hashtags = f"#{topic.replace(' ', '')} #AI #GeneratedContent"
    
    parsed_narration = [p for p in parsed_narration if p] # Remove empty strings
    parsed_visuals = [p for p in parsed_visuals if p]


    return parsed_narration, parsed_visuals, parsed_hashtags.strip()


In [6]:
# --- 3. AUDIO GENERATION ---
def generate_audio(text, tts_model, output_path, speaker_wav=None): # speaker_wav for XTTS
    print(f"Generating audio for: {text}")
    if "xtts" in TTS_MODEL_ID.lower() and speaker_wav:
        tts_model.tts_to_file(text, speaker_wav=speaker_wav, language="en", file_path=output_path)
    else: # For other models like Tacotron
        tts_model.tts_to_file(text, file_path=output_path)
    print(f"Audio saved to {output_path}")
    return output_path

In [7]:
# --- 4. VISUAL GENERATION ---
# Option A: Image (SDXL) then Video (SVD) - Recommended for "Subject to Video"
def generate_image_then_video(image_prompt, i2v_pipe, t2i_pipe, refiner_pipe, scene_idx):
    # Generate Image (Keyframe)
    print(f"Generating keyframe image for: {image_prompt}")
    # For SDXL, you can add negative prompts, specify num_inference_steps, guidance_scale etc.
    # Using SDXL
    image = t2i_pipe(
        prompt=image_prompt,
        # negative_prompt="low quality, blurry, watermark", # Example
        num_inference_steps=30, # SDXL typically needs fewer steps
        guidance_scale=7.5,
        # If using refiner:
        # output_type="latent" if refiner_pipe else "pil",
    ).images[0]
    # if refiner_pipe:
    #     image = refiner_pipe(prompt=image_prompt, image=image[None, :]).images[0]

    image_path = os.path.join(OUTPUT_DIR, f"scene_{scene_idx}_keyframe.png")
    image.save(image_path)
    print(f"Keyframe image saved to {image_path}")

    # Generate Video from Image (SVD)
    print(f"Generating video from image using SVD...")
    # SVD parameters
    video_frames = i2v_pipe(
        image,
        decode_chunk_size=8, # Lower if OOM, max typically 8 or 4
        num_frames=25, # SVD XT default, ~4 seconds at low FPS
        motion_bucket_id=127, # Adjust for more/less motion
        fps=7, # Low FPS for SVD output, will be re-timed in MoviePy
        noise_aug_strength=0.02 # Default
    ).frames[0]

    video_clip_path = os.path.join(OUTPUT_DIR, f"scene_{scene_idx}_svd.mp4")
    export_to_video(video_frames, video_clip_path, fps=7) # Save with SVD's native FPS
    print(f"SVD video clip saved to {video_clip_path}")
    return video_clip_path

# Option B: Direct Text-to-Video (ModelScope)
def generate_direct_video(video_prompt, t2v_pipe, scene_idx):
    print(f"Generating direct video for: {video_prompt}")
    # ModelScope parameters
    video_frames = t2v_pipe(video_prompt, num_inference_steps=25, num_frames=20).frames # ~2-3 sec
    video_clip_path = os.path.join(OUTPUT_DIR, f"scene_{scene_idx}_t2v.mp4")
    export_to_video(video_frames, video_clip_path, fps=8) # ModelScope native FPS
    print(f"T2V video clip saved to {video_clip_path}")
    return video_clip_path


In [8]:
# --- 5. VIDEO ASSEMBLY ---
def assemble_final_video(video_clip_paths, audio_clip_paths, narration_parts, output_filename="final_reel.mp4"):
    print("Assembling final video...")
    final_clips = []
    target_resolution = (1080, 1920) # Instagram Reel 9:16
    source_clips_to_close = []

    # Use a specific, existing font path
    font_path_for_textclip = "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf"
    # Fallback if the above is not found, try DejaVu:
    # if not os.path.exists(font_path_for_textclip):
    #     font_path_for_textclip = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
    # Fallback to Arial Bold from msttcorefonts if others aren't there:
    # if not os.path.exists(font_path_for_textclip):
    #     font_path_for_textclip = "/usr/share/fonts/truetype/msttcorefonts/arialbd.ttf" # or Arial_Bold.ttf

    # IMPORTANT: Check if the chosen font_path_for_textclip actually exists before proceeding
    if not os.path.exists(font_path_for_textclip):
        print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print(f"ERROR: Font file NOT FOUND at: {font_path_for_textclip}")
        print(f"Please verify the font path and filename.")
        print(f"Available font directories you listed: dejavu, droid, liberation, msttcorefonts, noto, ubuntu")
        print(f"Example: Check 'ls /usr/share/fonts/truetype/liberation/' to find the exact .ttf file name.")
        print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        # You might want to raise an error here or use a very generic font name that ImageMagick might find
        # For now, let it proceed, but Pillow will likely fail again if the path is wrong.
        # raise FileNotFoundError(f"Font file not found: {font_path_for_textclip}")


    for i, (video_path, audio_path, text_caption) in enumerate(zip(video_clip_paths, audio_clip_paths, narration_parts)):
        video_clip_temp = VideoFileClip(video_path)
        source_clips_to_close.append(video_clip_temp)

        video_clip_resized = video_clip_temp.resized(height=target_resolution[1])

        if video_clip_resized.w > target_resolution[0]:
            video_clip_final_shape = video_clip_resized.cropped(x_center=video_clip_resized.w/2, width=target_resolution[0])
        else:
            video_clip_final_shape = video_clip_resized

        video_clip_positioned = video_clip_final_shape.with_position('center')

        audio_clip_temp = AudioFileClip(audio_path)
        source_clips_to_close.append(audio_clip_temp)

        video_clip_with_audio = video_clip_positioned.with_audio(audio_clip_temp)
        video_clip_timed = video_clip_with_audio.with_duration(audio_clip_temp.duration)

        # Corrected TextClip call:
        txt_clip_temp = TextClip(
            font_path_for_textclip,    # First argument is font
            text=text_caption,         # 'text' is a keyword argument
            font_size=60,              # CORRECTED: font_size (with underscore)
            color='white',
            stroke_color='black',
            stroke_width=2,
            method='caption',
            size=(int(target_resolution[0]*0.8), None)
        )
        source_clips_to_close.append(txt_clip_temp)

        txt_clip_final = txt_clip_temp.with_position(('center', 0.8), relative=True).with_duration(video_clip_timed.duration)

        scene_composite = CompositeVideoClip([video_clip_timed, txt_clip_final], size=target_resolution)
        final_clips.append(scene_composite)

    if not final_clips:
        print("No clips to assemble!")
        for clip_to_close in source_clips_to_close:
            if hasattr(clip_to_close, 'close') and callable(clip_to_close.close):
                clip_to_close.close()
        return None

    final_video = concatenate_videoclips(final_clips, method="compose")
    final_video_path = os.path.join(OUTPUT_DIR, output_filename)
    
    try:
        final_video.write_videofile(final_video_path, 
                                    fps=24, 
                                    codec="libx264", 
                                    audio_codec="aac",
                                    threads=4, 
                                    preset="medium",
                                    logger='bar'
                                   ) 
    except Exception as e:
        print(f"Error during video writing: {e}")
        print("Make sure ffmpeg is correctly installed and accessible by MoviePy.")
        print(f"Ensure the font '{font_path_for_textclip}' is available or try a system default font.")
        import traceback
        traceback.print_exc()
        return None
    finally:
        for clip_to_close in source_clips_to_close:
            if hasattr(clip_to_close, 'close') and callable(clip_to_close.close):
                try:
                    clip_to_close.close()
                except Exception as e_close:
                    print(f"Error closing clip {type(clip_to_close)}: {e_close}")
        if hasattr(final_video, 'close') and callable(final_video.close):
            try:
                final_video.close()
            except Exception as e_close:
                print(f"Error closing final_video: {e_close}")

    print(f"Final video saved to {final_video_path}")
    return final_video_path

In [9]:
# --- MAIN WORKFLOW ---
def main_automation_flow(topic, use_svd_flow=True):
    # --- Load models ---
    # Manage VRAM: Load one by one or use .cpu_offload()
    llm = None
    tts_model = None
    t2i_pipe, refiner = None, None
    i2v_pipe = None
    t2v_pipe = None
    final_video_path = None
    speaker_reference_audio = "record_out.wav" # REQUIRED for XTTS voice cloning
    # Make sure speaker_reference_audio exists or provide a default way to handle TTS without it.
    # e.g. if not os.path.exists(speaker_reference_audio): speaker_reference_audio = None

    try:
        # 1. LLM for Script and Prompts
        llm_model, llm_tokenizer = load_llm() # Modified load_llm
        narration_scenes, visual_prompts_scenes, hashtags = generate_script_and_prompts_with_chat_template(topic, llm_model, llm_tokenizer)
        clear_vram(llm_model) # Unload LLM to free VRAM for vision models

        # 2. TTS for Narration
        tts_model = load_tts()
        audio_paths = []
        for i, scene_text in enumerate(narration_scenes):
            audio_file = os.path.join(OUTPUT_DIR, f"scene_{i}_audio.wav")
            # For XTTSv2, provide speaker_wav for voice cloning.
            # Ensure you have a reference audio file for XTTS if using it.
            # If speaker_reference_audio is not set, XTTS might use a default voice or fail.
            # For non-XTTS models, speaker_wav can be None.
            generate_audio(scene_text, tts_model, audio_file, speaker_wav=speaker_reference_audio if "xtts" in TTS_MODEL_ID.lower() else None)
            audio_paths.append(audio_file)
        clear_vram(tts_model)

        # 3. Visual Generation
        video_clip_paths = []
        if use_svd_flow: # Image (SDXL) -> Video (SVD)
            t2i_pipe, refiner = load_t2i_pipeline() # SDXL
            i2v_pipe = load_i2v_pipeline()         # SVD
            for i, visual_prompt in enumerate(visual_prompts_scenes):
                clip_path = generate_image_then_video(visual_prompt, i2v_pipe, t2i_pipe, refiner, i)
                video_clip_paths.append(clip_path)
            clear_vram(t2i_pipe, refiner, i2v_pipe)
        else: # Direct Text-to-Video (ModelScope)
            t2v_pipe = load_t2v_pipeline()
            for i, visual_prompt in enumerate(visual_prompts_scenes):
                clip_path = generate_direct_video(visual_prompt, t2v_pipe, i)
                video_clip_paths.append(clip_path)
            clear_vram(t2v_pipe)

        # 4. Video Assembly
        if video_clip_paths and audio_paths:
            final_video_path = assemble_final_video(video_clip_paths, audio_paths, narration_scenes)
        else:
            print("Not enough assets generated to assemble video.")

        # 5. Output final info
        if final_video_path:
            print("\n--- AUTOMATION COMPLETE ---")
            print(f"Final Video: {final_video_path}")
            print(f"Suggested Instagram Caption Text:\n{' '.join(narration_scenes)}")
            print(f"Suggested Hashtags: {hashtags}")
        else:
            print("\n--- AUTOMATION FAILED ---")
            print("Check logs for errors.")

    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()
    finally:
        # Ensure all models are cleared from VRAM if they were loaded
        print("Cleaning up any remaining models from VRAM...")
        models_to_clear = [m for m in [llm, tts_model, t2i_pipe, refiner, i2v_pipe, t2v_pipe] if m is not None]
        if models_to_clear:
            clear_vram(*models_to_clear)
        print("Cleanup finished.")


In [10]:
if __name__ == "__main__":
    # --- IMPORTANT SETUP ---
    # 1. Install dependencies:
    #    pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    #    pip install transformers accelerate bitsandbytes diffusers TTS moviepy Pillow safetensors sentencepiece
    #    (Make sure CUDA version in pytorch matches your system's CUDA Toolkit)
    # 2. Download models:
    #    The first time you run, Hugging Face models will download.
    #    For Coqui TTS, ensure you have the model files (XTTSv2 needs manual download or will try).
    # 3. For XTTSv2 voice cloning:
    #    Set `speaker_reference_audio` to a path of a clean WAV file of the voice you want to clone.
    #    It must be a 16-bit PCM WAV file, ideally >15 seconds long.
    # 4. Fonts for MoviePy: Make sure 'Arial-Bold' or your chosen font is available to MoviePy/ImageMagick.
    #    If not, ImageMagick might need to be installed and configured, or use a default font.

    # --- RUN THE SCRIPT ---
    # topic_for_reel = "The impact of AI on creative arts"
    topic_for_reel = "benefits of meditation."
    # topic_for_reel = "The secrets of black holes"

    # Set to True for SDXL -> SVD flow (better "subject" control)
    # Set to False for direct Text-to-Video (ModelScope, simpler but maybe less coherent)
    use_image_to_video_flow = True

    # Ensure you have a speaker reference audio file if using XTTSv2
    # Create a dummy file for testing if you don't have one, but quality will be default.
    if not os.path.exists("record_out.wav") and "xtts" in TTS_MODEL_ID.lower():
        print("WARNING: XTTSv2 speaker reference audio not found. TTS might use a default voice or fail.")
        print("Please set 'speaker_reference_audio' in main_automation_flow to a valid .wav file.")
        # As a placeholder, you might allow it to run with default (if model supports) or skip audio:
        # speaker_reference_audio = None # This might make XTTSv2 use its default speaker.

    main_automation_flow(topic_for_reel, use_svd_flow=use_image_to_video_flow)

Loading LLM...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generating script and prompts for topic (chat template): benefits of meditation.
LLM Response (model generated part only with chat template):
 Narration Script:
Scene 1: Close-up of a person meditating, eyes closed, with calming music in the background. Narration: "Feel the stress melt away with the power of meditation."
Scene 2: Time-lapse video of a busy city turning into a serene landscape as the person continues to meditate. Narration: "Experience inner peace and clarity, no matter the chaos around you."

Visual Prompts:
Scene 1: Soft focus, warm lighting, and a serene background with nature sounds.
Scene 2: Fast-paced video of the city with a calming voiceover and a peaceful background music.

Hashtags:
#meditation #mindfulness #selfcare #stressrelief #peaceandcalm #innerpeace #mindfulmoments #mindsetcoach #meditate #meditationcommunity #meditationforbeginners #meditationtime #meditationforanxiety #meditationforfocus #meditationforstudents #meditationforyoungadults #meditationfors

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


SVD video clip saved to instagram_content/scene_1_svd.mp4
VRAM cleared and memory collected.
Assembling final video...
{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'Lavf61.1.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1024, 576], 'bitrate': 360, 'fps': 7.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'Lavc61.3.100 libx264'}}], 'input_number': 0}], 'duration': 3.57, 'bitrate': 363, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [1024, 576], 'video_bitrate': 360, 'video_fps': 7.0, 'video_duration': 3.57, 'video_n_frames': 24}
/home/gowrav/Development/influencer/.venv/lib/python3.12/site

MoviePy - Done.
MoviePy - Writing video instagram_content/final_reel.mp4



frame_index:  11%|██                 | 87/803 [00:11<01:38,  7.26it/s, now=None]/home/gowrav/Development/influencer/.venv/lib/python3.12/site-packages/moviepy/video/io/ffmpeg_reader.py:178: UserWarning: In file instagram_content/scene_0_svd.mp4, 1769472 bytes wanted but 0 bytes read at frame index 25 (out of a total 24 frames), at time 3.57/3.57 sec. Using the last valid frame instead.
  warnings.warn(
frame_index:  11%|██▏                | 91/803 [00:12<01:37,  7.33it/s, now=None]/home/gowrav/Development/influencer/.venv/lib/python3.12/site-packages/moviepy/video/io/ffmpeg_reader.py:178: UserWarning: In file instagram_content/scene_0_svd.mp4, 1769472 bytes wanted but 0 bytes read at frame index 26 (out of a total 24 frames), at time 3.71/3.57 sec. Using the last valid frame instead.
  warnings.warn(
frame_index:  12%|██▏                | 94/803 [00:12<01:37,  7.28it/s, now=None]/home/gowrav/Development/influencer/.venv/lib/python3.12/site-packages/moviepy/video/io/ffmpeg_reader.py:178

MoviePy - Done !
MoviePy - video ready instagram_content/final_reel.mp4
Final video saved to instagram_content/final_reel.mp4

--- AUTOMATION COMPLETE ---
Final Video: instagram_content/final_reel.mp4
Suggested Instagram Caption Text:
Close-up of a person meditating, eyes closed, with calming music in the background. Narration: "Feel the stress melt away with the power of meditation." Time-lapse video of a busy city turning into a serene landscape as the person continues to meditate. Narration: "Experience inner peace and clarity, no matter the chaos around you."
Suggested Hashtags: #meditation #mindfulness #selfcare #stressrelief #peaceandcalm #innerpeace #mindfulmoments #mindsetcoach #meditate #meditationcommunity #meditationforbeginners #meditationtime #meditationforanxiety #meditationforfocus #meditationforstudents #meditationforyoungadults #meditationforstressrelief #meditationfordepression #meditationforinsomnia #meditationforbackpain #meditationforheadache #meditationforanxietya